<a href="https://colab.research.google.com/github/Achillesy/Fetal_Brain_GA_Prediction/blob/master/fmri_ga_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestational age prediction using fetal brain MRI
## Head circumference-based approach



## Please upload your Fetal Functional MRI files

This program allows users to upload multiple fMRI images at once and predict gestational age based on head circumference.

The predicted value is calculated using an empirical formula, which can be customized by the user.

The program code and predicted values are for research and academic purposes only. **They should not be used for commercial applications.**

![upload_guide](https://github.com/Achillesy/Fetal_Functional_MRI_Segmentation/blob/master/figures/upload_guide.png?raw=1)

1. Click the **Files** icon on the left
2. Click the **Upload to session storage** icon above
3. Your uploaded files will be displayed here

----
After double-checking your uploaded files, by <font color="green">pressing the **Enter** key in the input box below</font>, the fMRI mask will be automatically generated in a short time.

In [ ]:
input()

## Install ![monai](https://monai.io/assets/img/MONAI-logo_color.png) and check environment variables

In [ ]:
!pip install monai

from monai.config import print_config
print_config()


In [ ]:
import os
import numpy as np
import nibabel as nib
from glob import glob

import torch
from types import SimpleNamespace

cfg = SimpleNamespace(**{})
cfg.pixdim = (3.5, 3.5, 3.5)
cfg.roi_size = [64, 64, 64]
cfg.sw_batch_size = 4
cfg.file_pth = "fold4_train_metric_vnet.pth"

cfg.mri_dir = "mri"
cfg.mask_dir = "mask"
os.makedirs(cfg.mri_dir, exist_ok=True)
os.makedirs(cfg.mask_dir, exist_ok=True)

cfg.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Load VNet model and parameters

In [ ]:
!wget https://github.com/Achillesy/Fetal_Functional_MRI_Segmentation/releases/download/v1.0.0/fold4_train_metric_vnet.pth

In [ ]:
from monai.networks.nets import VNet

model = VNet(
  spatial_dims=3,
  in_channels=1,
  out_channels=2,
  act=("elu", {"inplace": True}),
  dropout_dim=3,
  bias=False,
).to(cfg.device)
model.load_state_dict(torch.load(cfg.file_pth, cfg.device))


In [ ]:
from monai.transforms import (
  AsDiscreted,
  Compose,
  EnsureChannelFirstd,
  Invertd,
  LoadImaged,
  NormalizeIntensityd,
  Orientationd,
  SaveImaged,
  Spacingd,
)

test_transforms = Compose(
  [
    LoadImaged(keys=["image"], image_only=True),
    EnsureChannelFirstd(keys=["image"]),
    Orientationd(keys=["image"], axcodes="RAS"),
    Spacingd(keys=["image"], pixdim=cfg.pixdim, mode="bilinear"),
    NormalizeIntensityd(keys="image", nonzero=True),
  ]
)

post_transforms = Compose(
  [
    AsDiscreted(keys="pred", argmax=True),
  ]
)

## Split the uploaded file into multiple 3D MRI files

In [ ]:
frmi_files = glob("*.nii.gz")
for fmri_data in frmi_files:
  fmri_data_name = os.path.basename(fmri_data).replace(".nii.gz", "")
  image = nib.load(fmri_data)
  data = image.get_fdata()
  if len(data.shape) != 4:
    raise ValueError("Invalid shape of fMRI file format. Expected 4D shape: [x, y, z, t]")
  channel_list = np.split(data, data.shape[-1], axis=-1)
  for i, channel in enumerate(channel_list):
    channel_image = nib.Nifti1Image(channel, image.affine)
    channel_file_name = os.path.join(cfg.mri_dir, f"{fmri_data_name}_{i+1}.nii.gz")
    nib.save(channel_image, channel_file_name)


In [ ]:
test_files = []
rmi_files = glob(os.path.join(cfg.mri_dir, "*.nii.gz"))
for f_file in rmi_files:
  test_files.append({"image": f_file})
# print(test_files)
for file in test_files:
    print(file['image'])

## Generate MRI Mask and Predict Gestational Age

In [ ]:
from monai.inferers import sliding_window_inference
from monai.data import DataLoader, Dataset, decollate_batch

test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1)


### Obtain GA correlation coefficients FOP, BPD, HC

Before making GA predictions, please make sure that the default orientation of the fMRI you upload is correct after MONAI conversion:
``` python
  Orientationd(keys=["image"], axcodes="RAS"),
  Spacingd(keys=["image"], pixdim=cfg.pixdim, mode="bilinear")

```
The direction is adjusted to RAS, and the pixdim is adjusted to (3.5, 3.5, 3.5)

Otherwise, open the debug code section and check the output image..


In [ ]:
import cv2

def get_GA_Coefficinet(mask_data):
  mask_data[mask_data != 0] = 1
  sum_z_mask = np.sum(mask_data, axis=(0, 1))
  z_index = np.argmax(sum_z_mask)
  ax_mask = mask_data[:, :, z_index]
  # ax_img  = img_data[:, :, z_index]
  contours, _ = cv2.findContours(ax_mask.astype("uint8"), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  max_area = 0
  for contour in contours:
    area = cv2.contourArea(contour)
    if area > max_area:
      max_area = area
      max_contour = contour
  hull = cv2.convexHull(max_contour)
  perimeter = cv2.arcLength(hull, True)
  rect = cv2.minAreaRect(max_contour)
  fod = max(rect[1])
  bpd = min(rect[1])

  c_X = rect[0][0]
  c_Y = rect[0][1]
  H = rect[1][0]
  W = rect[1][1]
  V = min(H, W)
  A = rect[2] - 90
  R = np.sqrt(W*W + H*H) /2
  alpha = np.arctan(W/H)
  theta = A*np.pi/180
  gamma = np.pi/2 - alpha + theta
  X = c_X - R*np.cos(gamma)
  Y = c_Y - R*np.sin(gamma)

  return z_index, fod, bpd, perimeter, X, Y, W, H, A, hull


### Customize the empirical formula for calculating gestational age based on head circumference (in-to-in)

In [ ]:
def get_GA_HC(hc):
    hc = (hc + 7) / 10
    ga = 0.0185 * hc * hc + 0.1713 * hc + 11.184
    return ga

### Traverse all 3D volumes in fMRI and predict GA by each axial section

In [ ]:
import re
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

normal_list = []
with torch.no_grad():
  for idx, test_data in enumerate(test_loader):
    ax_dict = {}
    test_inputs = test_data["image"].to(cfg.device)
    test_data["pred"] = sliding_window_inference(
      test_inputs, cfg.roi_size, cfg.sw_batch_size, model
    )
    test_data = [post_transforms(i) for i in decollate_batch(test_data)]
    z_index, fod, bpd, perimeter, X, Y, W, H, A, hull = get_GA_Coefficinet(test_data[0]['pred'][0].cpu().numpy())
    fmri_name = os.path.basename(test_files[idx]['image'])
    pattern = r"_(\d+)\.nii\.gz"
    ax_dict['FileName'] = re.sub(pattern, '', fmri_name)
    ax_dict['pixdim'] = 3.5
    ax_dict['Slice'] = z_index
    ax_dict['FOD']  = round(ax_dict['pixdim'] * fod, 1)
    ax_dict['BPD']  = round(ax_dict['pixdim'] * bpd, 1)
    ax_dict['HC']  = round(ax_dict['pixdim'] * perimeter, 1)
    ax_dict['GA'] = get_GA_HC(ax_dict['HC'])
    normal_list.append(ax_dict)

    # Debug
    # ax_image = test_data[0]['image'][0].cpu().numpy()[..., z_index]
    # ax_mask =  test_data[0]['pred'][0].cpu().numpy()[..., z_index]

    # rect = Rectangle((X, Y), W, H, angle=A, edgecolor='y', facecolor='none')
    # fig, ax = plt.subplots()
    # ax.imshow(ax_image * ax_mask, cmap="gray")
    # ax.plot(hull[:,0,0], hull[:,0,1], 'r', linewidth=1)
    # ax.add_patch(rect)
    # ax.set_title(f"{ax_dict['FileName']}\nFOD={ax_dict['FOD']:.0f}mm BPD={ax_dict['BPD']:.0f}mm HC={ax_dict['HC']:.0f}mm GA={ax_dict['GA']:.3f}weeks")
    # plt.show()


## Estimated gestational age result

In [ ]:
import pandas as pd

normal_df = pd.DataFrame.from_dict(normal_list)
grouped = normal_df.groupby('FileName')
for key, group in grouped:
  print(key, end=": ")
  median_ga = group['GA'].median()
  week = int(median_ga)
  day = int((median_ga - week) * 7)
  print(f"{week} weeks {day} days")

# Debug
# display(normal_df)